In [ ]:
!python train_val_test_split.py

In [4]:
# Create CSV data files and TFRecord files
!python3 xml_to_csv.py

!python3 generate_tfrecord.py --csv_input=data/train_labels.csv --labelmap=data/labelmap.txt --image_dir=images/train --output_path=data/train.tfrecord
!python3 generate_tfrecord.py --csv_input=data/validation_labels.csv --labelmap=data/labelmap.txt --image_dir=images/validation --output_path=data/val.tfrecord

Processing /Users/tanimsk/Programming/ML/tensorflow/workspace/object-detection/images/train

------------------------------------
Invalid bounding box in /Users/tanimsk/Programming/ML/tensorflow/workspace/object-detection/images/train/kandalaksha-russia-may-24-2018-260nw-1127352350_jpg.rf.fdae0df5dbd1a776e06da979b8ca1717.xml
------------------------------------
Invalid bounding box in /Users/tanimsk/Programming/ML/tensorflow/workspace/object-detection/images/train/pic_131_jpg.rf.d34da2f2b1c91f219f20b7afd2dbd21d.xml
------------------------------------
Invalid bounding box in /Users/tanimsk/Programming/ML/tensorflow/workspace/object-detection/images/train/video16_520_jpg.rf.0bdee584d6cd2ae9e889903a4c52d193.xml
------------------------------------
Invalid bounding box in /Users/tanimsk/Programming/ML/tensorflow/workspace/object-detection/images/train/video19_1889_jpg.rf.4e17bb5728ea3a8a3cd603482280b725.xml
------------------------------------
Invalid bounding box in /Users/tanimsk/Progra

In [5]:
train_record_fname = 'data/train.tfrecord'
val_record_fname = 'data/val.tfrecord'
label_map_pbtxt_fname = 'data/labelmap.pbtxt'

# 4.&nbsp;Set Up Training Configuration

In [6]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },    
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

## Download the model, if not downloaded

In [ ]:
# Create "mymodel" folder for holding pre-trained weights and configuration files
%mkdir pre-trained-models
%cd pre-trained-models

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!curl -O {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!curl -O {download_config}

%cd ..

In [7]:
# Set training parameters for the model
num_steps = 30000

if chosen_model == 'efficientdet-d0':
  batch_size = 4
else:
  batch_size = 16

In [8]:
# Set file locations and get number of classes for config file
pipeline_fname = "pre-trained-models/" + base_pipeline_file
fine_tune_checkpoint = "pre-trained-models/" + model_name + "/checkpoint/ckpt-0"


def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util

    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True
    )
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())


num_classes = get_num_classes(label_map_pbtxt_fname)
print("Total classes:", num_classes)

Total classes: 3


## Writing the config file

In [9]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re
import os

print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()

os.makedirs(f"models/{model_name}", exist_ok=True)
with open(f'models/{model_name}/pipeline.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)

!cat models/{model_name}/pipeline.config

writing custom configuration file
# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
 

In [10]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = f"models/{model_name}/pipeline.config"
model_dir = f"models/{model_name}"

# Train the model   

In [11]:
# Run training!
!python model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

2024-11-26 08:14:09.315006: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-11-26 08:14:09.315050: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-11-26 08:14:09.315056: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-11-26 08:14:09.315532: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-26 08:14:09.315945: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1126 08:14:09.323697 8717307712 mirrored_strategy.py:419] Using MirroredStrategy wit